In [1]:
from torch import load
from importlib import reload


In [2]:
import model
reload(model)
Sequential = model.Sequential
ReLU = model.ReLU
Conv2d = model.Conv2d
NNUpsampling = model.NNUpsampling
Sigmoid = model.Sigmoid
MSE = model.MSE
Upsampling = model.Upsampling


# Load training data

In [3]:
# Load train data
TRAIN_PATH = '../data/train_data.pkl'
train_noisy_imgs_input, train_noisy_imgs_target = load(TRAIN_PATH)
train_noisy_imgs_input = train_noisy_imgs_input/255
train_noisy_imgs_target = train_noisy_imgs_target/255
training_set_size,num_channels,y_size,x_size = train_noisy_imgs_input.shape

In [8]:
class Model():
    def __init__(self, in_channels=3, out_channels=3, lr=1e-3):
        # instantiate model + optimizer + loss function + any other stuff you need
        self.model = Sequential(
                        Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, stride=2, padding=1),
                        ReLU(),
                        Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1),
                        ReLU(),
                        Upsampling(scale_factor=2, in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),
                        ReLU(),
                        Upsampling(scale_factor=2, in_channels=16, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                        Sigmoid()
                        )
        self.criterion = MSE()
        self.lr = lr

    def load_pretrained_model(self):
        ## This loads the parameters saved in bestmodel.pth into the model
        pass

    def train(self, train_input, train_target):
        # train ̇input: tensor of size (N, C, H, W) containing a noisy version of the images.
        # train target: tensor of size (N, C, H, W) containing another noisy version of the
        # same images, which only differs from the input by their noise.
        batch_size = 32
        nb_epochs = 3
        self.losses = []
        avg_loss = 0

        for e in range(nb_epochs):
            print('Doing epoch %d'%e)
            for b in range(0, train_input.size(0), batch_size):
                if b % 5 == 0:
                    self.losses.append(avg_loss/5)
                    avg_loss = 0
                if b%50 ==0:
                    print(self.losses[-1])
                # forward pass
                output = self.model(train_input[b:b+batch_size])
                loss = self.criterion(output, train_target[b:b+batch_size])
                avg_loss+=loss.item()
                # make step
                self.model.zero_grad()
                gradient = self.criterion.backward()
                gradient = self.model.backward(gradient)
                for (parameter,_) in self.model.param():
                    parameter -= parameter.grad*self.lr

    def predict(self, test_input):
        #:test ̇input: tensor of size (N1, C, H, W) that has to be denoised by the trained
        # or the loaded network.
        #: returns a tensor of the size (N1, C, H, W)
        pass

In [9]:
ourModel = Model()
ourModel.train(train_noisy_imgs_input, train_noisy_imgs_target)

Doing epoch 0
0.0
0.06268877759575844
0.05846758931875229
0.0418437659740448
0.03524077385663986
0.0354213535785675
0.03451731503009796
0.025138859078288077
0.02949148416519165
0.037558384239673615
0.022657349333167077
0.027885977178812027
0.027602728083729745
0.025547520816326143
0.02397290915250778
0.02739260345697403
0.0215676698833704
0.023305001854896545
0.025292449817061424
0.02468079961836338
0.023063242062926294
0.02343892939388752
0.021910065785050392
0.022892505675554276
0.022123095765709876
0.022032814845442773
0.02177177295088768
0.023371782898902894
0.022373880073428155
0.021278548613190652


KeyboardInterrupt: 

In [ ]:
ourModel.losses

In [ ]:
y = [x,x,x,x]

In [ ]:
y

In [ ]:
flatten = lambda deep_list: [item for sublist in deep_list for item in sublist]

In [ ]:
flatten(y)

In [ ]:
x = zeros(3,4)

In [ ]:
x

In [ ]:
print('before: ',x.shape)